# Week 8
# Geospatial Plotting with Folium

Folium is a Python wrapper for leaflet.js, a JavaScript library for creating interactive maps. Today we will learn how to use Folium to visualize geospatial data.

In [ ]:
# install Folium
!pip install folium

In [ ]:
import folium
import numpy as np
import pandas as pd
folium.__version__

In [ ]:
# To create a base map, simply pass your starting coordinates to Folium:
m = folium.Map(location=[45.5236, -122.6750])

In [ ]:
m

In [ ]:
# Save it in a file.
m.save('index.html')

The default tiles are set to `OpenStreetMap`, but `Stamen Terrain`, `Stamen Toner`, `Mapbox Bright`, and `Mapbox Control Room`, and many others tiles are built in.

In [ ]:
folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=12
)

In [ ]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=11,
    tiles='Stamen Terrain'
)

tooltip = 'Click me!'

folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>', tooltip=tooltip).add_to(m)

m

## Exercise
Create a map of New York City, and mark the location of the Statue of Liberty, Empire State Building, and Lehman College.

Leaflet’s `Circle` and `CircleMarker`, implemented to reflect radii in units of meters and pixels respectively.

In [ ]:
m = folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


m

## Exercise
On the map of NYC, circle the airports.

## Choropleth Maps
A **choropleth map** is a type of thematic map in which areas are shaded or patterned in proportion to a statistical variable.

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json' # formatted string
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

state_data.head()

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

## Exercise
Create a Choropleth map that indicate the size of population of each state.

## Case Study: Visualizing Bus Traffic in NYC
Due to its high population, New York has now become the third most congested city in the world in terms of traffic and the second-worst in the United States. Therefore, it becomes important for the government and road planning authorities to do the analysis of traffic on roads. In this project, we’ll visualize the bus traffic in New York using Folium.

The NYC bus dataset can be found [here](https://www.kaggle.com/datasets/stoney71/new-york-city-transport-statistics?select=mta_1706.csv). It contains the latitude and longitude of buses recorded at an interval of every 10 minutes. Please download `mta_1706.csv`, which contains data recorded in June, 2017.

[Reference](https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/)

In [ ]:
# Decompress the downloaded file
import zipfile
f = zipfile.ZipFile("mta_1706.csv.zip", "r")
f.printdir()
f.extractall()
f.close()

In [ ]:
import pandas as pd
df = pd.read_csv("mta_1706.csv", on_bad_lines='skip')
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# Extract data for June 1st, 2017.
df = df[df['RecordedAtTime'].str.split(' ').apply(lambda x:x[0]=='2017-06-01')]
df.shape

In [ ]:
# We only need information such as data recording time, latitude,
# and longitude of buses and vehicle reference numbers for selecting
# a vehicle.
df = df[['RecordedAtTime','VehicleRef','VehicleLocation.Latitude','VehicleLocation.Longitude']]
df.head()

In [ ]:
# Check and drop duplicated records
df = df.drop_duplicates()
df.shape

In [ ]:
# Check for incomplete records
df.isnull().sum()

In [ ]:
# Converting column to datetime
df['RecordedAtTime'] = pd.to_datetime(df['RecordedAtTime'])
# Creating hour column
df['hour'] = df['RecordedAtTime'].dt.hour
df.head()

In [ ]:
# There is a problem here – not all buses have their locations
# recorded at the same point in time. Let's take the last recorded
# position of every bus in an hour.

hours = range(24)
vehicles = df['VehicleRef'].unique()
print(len(vehicles), vehicles)

In [ ]:
rows = []
for hour, vehicle in zip(hours, vehicles):
    temp_df = df[(df['hour'] == hour) & (df['VehicleRef'] == vehicle)].copy()
    temp_df.sort_values(['RecordedAtTime'], ascending=False, inplace=True)
    if temp_df.shape[0] != 0:
        row_index = temp_df.index.values[0]
        rows.append(row_index)
df_small = df.loc[rows, :]

In [ ]:
df_small.head()

In [ ]:
# Complete the data processing with `groupby` and `merge` (will be
# introduced later)
df2=pd.DataFrame(df.groupby(['hour','VehicleRef'])['RecordedAtTime'].max())
df2.reset_index(inplace=True)
df2.head()

In [ ]:
df3=pd.merge(df2,df,left_on=['hour','VehicleRef','RecordedAtTime'],right_on=['hour','VehicleRef','RecordedAtTime'])
df3.head()

In [ ]:
lat_long_list = []
for i in range(1,25):
    temp=[]
    for index, instance in df3[df3['hour'] == i].iterrows():
        temp.append([instance['VehicleLocation.Latitude'],instance['VehicleLocation.Longitude']])
    lat_long_list.append(temp)

In [ ]:
print(lat_long_list)

In [ ]:
from folium.plugins import HeatMapWithTime
m7=folium.Map(location=[40.712776, -74.005974],zoom_start=10)
m7

In [ ]:
# Create a heapmap with the bus traffics in NYC at different hours
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright').add_to(m7)
m7

In [ ]:
m7.save('m7.html')